In [15]:
!pip install keras-tuner #tuner helps you to pick up optimal set of hyperparamters 

     |████████████████████████████████| 61kB 3.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=66954880638568e06aa1ad1cf29e11c71bf5cbf9f44ed6037fc3fcb067dbd62e
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=4dbe0405dfe450f54ca3395a2c6d1ef5b8649b82d06b2fca2d0f8381e22aefc5
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [16]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [17]:
print(tf.__version__) #check for version - only tensorflow above 2.0 version supports keras-tuner

2.3.0


In [18]:
fashion_mnist=keras.datasets.fashion_mnist #dataset 

In [19]:
(train_img, train_label),(test_img,test_label)=fashion_mnist.load_data() #loading dataset into train and test sets

In [20]:
train_img=train_img/255.0  # for converting rgb values btw 0-255 into range of 0 to 1
test_img=test_img/255.0

In [21]:
train_img[0].shape

(28, 28)

In [22]:
test_img[0].shape

(28, 28)

In [23]:
train_img=train_img.reshape(len(train_img),28,28,1)
test_img=test_img.reshape(len(test_img),28,28,1)

In [24]:
def build_model(hp): #hp is hyperparameter which is managed by tuner 
  model=keras.Sequential([
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32, max_value=128, step=16) ,#filter to be used for convolution with values ranging from 32 to 128 
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]), # filter size ranging from 3 to 5
                              activation='relu' ,# activation layer for first nn is relu
                              input_shape=(28,28,1) # size of input images 
                          ),
                          keras.layers.Conv2D(
                              filters=hp.Int('conv_1_filter',min_value=32, max_value=64, step=16),
                              kernel_size=hp.Choice('conv_1_kernel',values=[3,5]) ,
                              activation='relu'
                              
                          ),
                          keras.layers.Flatten(), 
                          keras.layers.Dense( #dense layer is used 
                              units=hp.Int('dense_1_filter',min_value=32, max_value=128,step=16),
                              activation='relu'
                          ),
                          keras.layers.Dense(10, activation='softmax') #10 output nodes 


  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])), #choice will select value between le-2 and le-3
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy']
                )
  return model 

In [25]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters #used for tuning the parameters

In [26]:
tuner_search = RandomSearch(build_model, objective='val_accuracy',max_trials=5,directory= 'output',project_name='Mnist Fashion') #validation accuracy is used as objective
#trials shows us filter size used in some particular conv layer and what is the value of it 

In [27]:
tuner_search.search(train_img, train_label,epochs=3,validation_split=0.1) #splits the train_data into validation and train data 
#runtime should be in GPU 

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5108 - accuracy: 0.8183 - val_loss: 0.4018 - val_accuracy: 0.8550
Epoch 2/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3709 - accuracy: 0.8645 - val_loss: 0.3662 - val_accuracy: 0.8655
Epoch 3/3
1688/1688 [==============================] - 8s 5ms/step - loss: 0.3396 - accuracy: 0.8768 - val_loss: 0.4009 - val_accuracy: 0.8553


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.4489 - accuracy: 0.8399 - val_loss: 0.3567 - val_accuracy: 0.8683
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3218 - accuracy: 0.8829 - val_loss: 0.3379 - val_accuracy: 0.8783
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2830 - accuracy: 0.8944 - val_loss: 0.3370 - val_accuracy: 0.8857


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5649 - accuracy: 0.8028 - val_loss: 0.3944 - val_accuracy: 0.8540
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.4049 - accuracy: 0.8491 - val_loss: 0.4033 - val_accuracy: 0.8493
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3768 - accuracy: 0.8576 - val_loss: 0.4017 - val_accuracy: 0.8562


Epoch 1/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3696 - accuracy: 0.8671 - val_loss: 0.2746 - val_accuracy: 0.9005
Epoch 2/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.2270 - accuracy: 0.9169 - val_loss: 0.2421 - val_accuracy: 0.9062
Epoch 3/3
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1587 - accuracy: 0.9409 - val_loss: 0.2361 - val_accuracy: 0.9160


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.5360 - accuracy: 0.8128 - val_loss: 0.4107 - val_accuracy: 0.8452
Epoch 2/3
1688/1688 [==============================] - 6s 4ms/step - loss: 0.3939 - accuracy: 0.8535 - val_loss: 0.4539 - val_accuracy: 0.8363
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3686 - accuracy: 0.8636 - val_loss: 0.4074 - val_accuracy: 0.8463


INFO:tensorflow:Oracle triggered exit


In [28]:
model=tuner_search.get_best_models(num_models=1)[0] #searches for the best model 

In [29]:
model.summary() #description of model which is best for the dataset

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 96)        83040     
_________________________________________________________________
flatten (Flatten)            (None, 55296)             0         
_________________________________________________________________
dense (Dense)                (None, 96)                5308512   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                970       
Total params: 5,393,482
Trainable params: 5,393,482
Non-trainable params: 0
_________________________________________________________________


In [30]:
model.fit(train_img, train_label, epochs=10, validation_split=0.1, initial_epoch=3) # fit the model which is selected to train the dataset accurately
# starting epoch from 3 

Epoch 4/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.1067 - accuracy: 0.9603 - val_loss: 0.2854 - val_accuracy: 0.9153
Epoch 5/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0711 - accuracy: 0.9734 - val_loss: 0.3223 - val_accuracy: 0.9117
Epoch 6/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0484 - accuracy: 0.9822 - val_loss: 0.3501 - val_accuracy: 0.9153
Epoch 7/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0347 - accuracy: 0.9876 - val_loss: 0.3945 - val_accuracy: 0.9103
Epoch 8/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0260 - accuracy: 0.9909 - val_loss: 0.4738 - val_accuracy: 0.9130
Epoch 9/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0211 - accuracy: 0.9925 - val_loss: 0.5068 - val_accuracy: 0.9115
Epoch 10/10
1688/1688 [==============================] - 11s 6ms/step - loss: 0.0208 - accuracy: 0.9930 - val_loss: 0.5361 - val_a